In [1]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [2]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

In [3]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

### Eklima data

In [4]:
eklima_f = dict()
eklima_double_fence_acc = dict()
eklima_wind_dir_10m   = dict()
eklima_wind_speed_10m = dict()
eklima_air_temp = dict()
eklima_diff = dict()
eklima_westerly_idx = dict()
eklima_easterly_idx = dict()
eklima_westerly = dict()
eklima_easterly = dict()
eklima_total_precip = dict()
eklima_precip_west  = dict()
eklima_precip_east  = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        eklima_dirnc = '../../Data/Eklima/%s/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        eklima_f[int(year+month+day)] = netCDF4.Dataset(eklima_dirnc, 'r')

        eklima_double_fence_acc[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['RR_1'][:])
        eklima_wind_dir_10m[int(year+month+day)]   = (eklima_f[int(year+month+day)].variables['DD'][:])
        eklima_wind_speed_10m[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['FF'][:])
        eklima_air_temp[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['TA'][:])
        
        eklima_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        eklima_diff[int(year+month+day)] = np.empty(shape=(eklima_double_fence_acc[int(year+month+day)].shape))#/60))
        eklima_diff[int(year+month+day)][:] =  np.nan

        eklima_diff[int(year+month+day)][:,] = eklima_double_fence_acc[int(year+month+day)]

        # Find wind regimes and associated precip amount
        IDX = np.arange(eklima_wind_dir_10m[int(year+month+day)].shape[0])

        ## find all eklima_westerly regimes (202.5 - 22.5 deg) 
        eklima_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 202.5),
                                                                                     (eklima_wind_dir_10m[int(year+month+day)][:,] <= 360)),
                                                                      (eklima_wind_dir_10m[int(year+month+day)][:,] <22.5))]

        ## find all eklima_easterly regimes (22.5 - 202.5 deg)
        eklima_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 22.5),
                                                                       (eklima_wind_dir_10m[int(year+month+day)][:,] < 202.5))]

        eklima_westerly[int(year+month+day)] = eklima_westerly_idx[int(year+month+day)]
        eklima_easterly[int(year+month+day)] = eklima_easterly_idx[int(year+month+day)]

        if len(eklima_westerly[int(year+month+day)]) == 0:    # only easterlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(len(eklima_westerly[int(year+month+day)]),2))

        elif len(eklima_easterly[int(year+month+day)]) == 0:  # only westerlies
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(len(eklima_easterly[int(year+month+day)]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))
        else:    # both easterlies and westerlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)
            
#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))

        # assign sum values to days
        if len(eklima_easterly[int(year+month+day)]) == 0 and len(eklima_westerly[int(year+month+day)]) == 0:
            eklima_total_precip[int(year+month+day)] = np.nan
            eklima_precip_east[int(year+month+day)] = np.nan
            eklima_precip_west[int(year+month+day)] = np.nan
        else:
            eklima_total_precip[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
                                           np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],])
            eklima_precip_east[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],])
            eklima_precip_west[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],])


### Minuetly data

In [6]:
obs_f = dict()
obs_double_fence_acc_mean = dict()
obs_wind_dir_10m   = dict()
obs_wind_speed_10m = dict()
obs_double_fence_temp = dict()
obs_air_temp = dict()
obs_dofe_acc   = dict()
obs_wind_dir   = dict()
obs_wind_speed = dict() 
obs_dofe_acc_rolling = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_diff = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()
obs_total_precip = dict()
obs_precip_west  = dict()
obs_precip_east  = dict()
obs_air_temp_rolling = dict()

for month in m:
    if month == '11':
        t = np.arange(7,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        obs_dirnc = '../../Data/Weathermast/Met_%s%s%s_%s.nc' %(year,month,day,station)
        obs_f[int(year+month+day)] = netCDF4.Dataset(obs_dirnc, 'r')

        obs_double_fence_acc_mean[int(year+month+day)] = (np.nanmean(obs_f[int(year+month+day)].variables['double_fence_accum'][:], axis=1))
        obs_wind_dir_10m[int(year+month+day)]   = (obs_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
        obs_wind_speed_10m[int(year+month+day)] = (obs_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        obs_double_fence_temp[int(year+month+day)] = (obs_f[int(year+month+day)].variables['air_temp'][:])
        
        obs_f[int(year+month+day)].close
        
        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        if year == '2016' and month == '11' and day == '07':
            continue
        else:
            pp = (np.concatenate((obs_double_fence_acc_mean[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_acc_mean[int(year+month+day)][:])))
            obs_dofe_acc[int(year+month+day)] = (pp)
            wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[int(year+month+day)][:])))
            obs_wind_dir[int(year+month+day)] = (wd)
            ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[int(year+month+day)][:])))
            obs_wind_speed[int(year+month+day)] = (ws)
            
            at = (np.concatenate((obs_double_fence_temp[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_temp[int(year+month+day)][:])))
            obs_air_temp[int(year+month+day)] = at
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_dofe_acc[int(year+month+day)] = pd.DataFrame(np.array(obs_dofe_acc[int(year+month+day)])[:])
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc[int(year+month+day)].rolling(window=10,
                                                                                              min_periods= 7,
                                                                                              axis=0).mean()
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc_rolling[int(year+month+day)].values
        obs_dofe_acc_rolling[int(year+month+day)] = np.array(obs_dofe_acc_rolling[int(year+month+day)])[9:,:]
        ###
        obs_wind_dir[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_dir[int(year+month+day)])[:])
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir[int(year+month+day)].rolling(window=10,
                                                                                              min_periods=7,
                                                                                              axis=0).mean()
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir_rolling[int(year+month+day)].values
        obs_wind_dir_rolling[int(year+month+day)] = np.array(obs_wind_dir_rolling[int(year+month+day)])[9:,:]
        ###
        obs_wind_speed[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_speed[int(year+month+day)])[:])
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed_rolling[int(year+month+day)].values
        obs_wind_speed_rolling[int(year+month+day)] = np.array(obs_wind_speed_rolling[int(year+month+day)])[9:,:]
        ###
        obs_air_temp[int(year+month+day)] = pd.DataFrame(np.array(obs_air_temp[int(year+month+day)])[:])
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp_rolling[int(year+month+day)].values
        obs_air_temp_rolling[int(year+month+day)] = np.array(obs_air_temp_rolling[int(year+month+day)])[9:,:]
        
        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        obs_diff[int(year+month+day)] = np.empty(shape=(obs_dofe_acc_rolling[int(year+month+day)].shape))#/60))
        obs_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,obs_dofe_acc_rolling[int(year+month+day)].shape[0]-1):#/60):
            obs_diff[int(year+month+day)][int(i),:] = obs_dofe_acc_rolling[int(year+month+day)][int(i)+1] - \
                                                      obs_dofe_acc_rolling[int(year+month+day)][int(i)]


        # Find wind regimes and associated precip amount
        IDX = np.arange(obs_wind_dir_rolling[int(year+month+day)].shape[0])

        ## find all obs_westerly regimes (202.5 - 22.5 deg) 
        obs_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 202.5), 
                                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <= 360)),
                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <22.5))]
        ## find all obs_easterly regimes (22.5 - 202.5 deg)
        obs_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 22.5),
                                                                (obs_wind_dir_rolling[int(year+month+day)][:,0] < 202.5))]

        ## find indices for all obs_westerly regimes 
        sub_idx = []

        for i in range(0,obs_westerly_idx[int(year+month+day)] .shape[0]-1):
            if obs_westerly_idx[int(year+month+day)] [i]+1 == obs_westerly_idx[int(year+month+day)] [i+1]:
                continue
            else:
                #print(i)
                sub_idx.append(i)
    
    #        print([int(year+month+day)], obs_westerly_idx[int(year+month+day)], sub_idx)
        
        
        ## find westerlies which are < 29 min and assign them to obs_easterly wind regime
        if len(obs_westerly_idx[int(year+month+day)]) == 0 and len(sub_idx) == 0:     # westerlies do not exist
            obs_westerly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
#            print('No obs_westerly wind on %s' %(int(year+month+day)))
        elif len(obs_westerly_idx[int(year+month+day)]) != 0 and len(sub_idx) == 0:   # westerlies exist only for one time period a day
            obs_westerly[int(year+month+day)] = []
      #      obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
            
            obs_westerly_duration = obs_westerly_idx[int(year+month+day)][-1] - obs_westerly_idx[int(year+month+day)][0]
            if obs_westerly_duration >= 28:
                obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
            elif obs_westerly_duration < 28:
                obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
        else:
            obs_westerly[int(year+month+day)] = []
         #   obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):                # westerlies and easterlies interchange daily
                if i == 0:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[0]] - \
                                         obs_westerly_idx[int(year+month+day)] [0])
                elif i == np.array(sub_idx).shape[0]:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [-1] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[i]] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1])
                if obs_westerly_duration >= 28:
                    if i == 0:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
            #            print(obs_westerly_duration, RR_obs_westerly)
                    elif i == np.array(sub_idx).shape[0]:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
             #           print(obs_westerly_duration, RR_obs_westerly)
                    else:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
              #          print(obs_westerly_duration, RR_obs_westerly)
                elif obs_westerly_duration < 28:
                    if i == 0:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(obs_westerly_duration)
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
        

        if len(obs_westerly[int(year+month+day)]) == 0:    # only easterlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)


        elif len(obs_easterly[int(year+month+day)]) == 0:  # only westerlies
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)

        else:    # both easterlies and westerlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)
            

            
        # assign sum values to days
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip[int(year+month+day)] = np.nan
            obs_precip_east[int(year+month+day)] =  np.nan
            obs_precip_west[int(year+month+day)] =  np.nan
        else:
            obs_total_precip[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
                                                np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:])
            obs_precip_east[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            
#        print([int(year+month+day)], 'total:', np.round(obs_total_precip[int(year+month+day)],2),
 #                                       'westerly:',  np.round(obs_precip_west[int(year+month+day)],2),
  #                                       'easterly:', np.round(obs_precip_east[int(year+month+day)],2) )

############################            
# assign eklima data where total precip < 0
        if obs_total_precip[int(year+month+day)] < 0:
            if obs_precip_east[int(year+month+day)] < 0 and obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
    
            elif obs_precip_east[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
        
            elif obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
        
        elif obs_precip_east[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])   
    
        elif obs_precip_west[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
        elif np.isnan(obs_total_precip[int(year+month+day)]):
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            


### Retrieval data

In [7]:
txt_dir = '../../Data/Retrieval/'


In [8]:
for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        Date = int(year+month+day)
        
        
        if len(obs_westerly[Date]) != 0:
            # use pulsed file
            txt_filename = 'pulsed_minute_surface_acc_Haukeli.dat.txt'
            Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
            idx = np.where(Haukeli['Unnamed: 0'] == Date)
            if np.array(idx).shape[1] == 1:
                precip_24 = (Haukeli['Unnamed: 1'][idx[0][0]])
                precip_24 = np.array([precip_24])
                
                #read in the surface snowfall rate for each minute
                column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                column0 = column0.convert_objects(convert_numeric=True)
                column1 = column1.convert_objects(convert_numeric=True)
                column2 = column2.convert_objects(convert_numeric=True)
                column3 = column3.convert_objects(convert_numeric=True)
                column4 = column4.convert_objects(convert_numeric=True)
                column5 = column5.convert_objects(convert_numeric=True)
                
                ret_pulsed = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                ret_pulsed = np.array(ret_pulsed)
                ret_pulsed = ret_pulsed.flatten()/ret_pulsed.flatten().shape[0]
                
                if len(ret_pulsed) == len(obs_diff[Date]):
                    ret_pulsed_24 = np.empty(shape=ret_pulsed.shape)
                    ret_pulsed_24[:] = np.nan
                    ret_pulsed_24[:] = ret_pulsed[:]

                elif len(ret_pulsed) != len(obs_diff[Date]):
                    ret_pulsed_24 = np.empty(shape=(int(ret_pulsed.shape[0]/60), ))
                    ret_pulsed_24[:] = np.nan
                    for i in range(int(ret_pulsed.shape[0]/60)):
                        ret_pulsed_24[i] = np.sum(ret_pulsed[i*60:(i+1)*60])
                
                del ret_pulsed    
          
                # assign precip values to westerly wind regime easterly wind regime will have nan
                ret_precip = np.empty(shape=ret_pulsed_24.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_westerly[Date],] = ret_pulsed_24[obs_westerly[Date],]
                    
                ret_upslope_24 = np.empty(shape=ret_pulsed_24.shape)
                ret_upslope_24[:] = np.nan
                
                # then where it is nan assign easterly values
                if len(obs_easterly[Date]) != 0:
                    # use upslope file
                    txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'
                    Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
                    idx = np.where(Haukeli['Unnamed: 0'] == Date)
                    
                    column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                    column0 = column0.convert_objects(convert_numeric=True)
                    column1 = column1.convert_objects(convert_numeric=True)
                    column2 = column2.convert_objects(convert_numeric=True)
                    column3 = column3.convert_objects(convert_numeric=True)
                    column4 = column4.convert_objects(convert_numeric=True)
                    column5 = column5.convert_objects(convert_numeric=True)

                    ret_upslope = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                    ret_upslope = np.array(ret_upslope)

                    ret_upslope = ret_upslope.flatten()/ret_upslope.flatten().shape[0]
                    
                    if len(ret_upslope) == len(obs_diff[Date]):
                        ret_upslope_24 = np.empty(shape=ret_upslope.shape)
                        ret_upslope_24[:] = np.nan
                        ret_upslope_24[:] = ret_upslope[:]
                    elif len(ret_upslope) != len(obs_diff[Date]):
                        ret_upslope_24 = np.empty(shape=(int(ret_upslope.shape[0]/60), ))
                        ret_upslope_24[:] = np.nan
                        for i in range(int(ret_upslope.shape[0]/60)):
                            ret_upslope_24[i] = np.sum(ret_upslope[i*60:(i+1)*60])
                    del ret_upslope
                    
                    #assing precip values to easterly wind regime
                    ret_precip[obs_easterly[Date],] = ret_upslope_24[obs_easterly[Date],]
            if np.array(idx).shape[1] == 0:
                print([Date], 'no retried values')                    
                ret_upslope_24 = np.empty(shape=obs_diff[Date].shape)
                ret_upslope_24[:] = np.nan
                
                ret_pulsed_24 = np.empty(shape=obs_diff[Date].shape)
                ret_pulsed_24[:] = np.nan
                
                ret_precip = np.empty(shape=obs_diff[Date].shape)
                ret_precip[:] = np.nan
                
        elif len(obs_westerly[Date]) == 0 and len(obs_easterly[Date]) != 0:
            # use upslope file
            txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'
            Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
            idx = np.where(Haukeli['Unnamed: 0'] == Date)

            if np.array(idx).shape[1] == 1:
                column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                column0 = column0.convert_objects(convert_numeric=True)
                column1 = column1.convert_objects(convert_numeric=True)
                column2 = column2.convert_objects(convert_numeric=True)
                column3 = column3.convert_objects(convert_numeric=True)
                column4 = column4.convert_objects(convert_numeric=True)
                column5 = column5.convert_objects(convert_numeric=True)

                ret_upslope = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                ret_upslope = np.array(ret_upslope)

                ret_upslope = ret_upslope.flatten()/ret_upslope.flatten().shape[0]

                if len(ret_upslope) == len(obs_diff[Date]):
                    ret_upslope_24 = np.empty(shape=ret_upslope.shape)
                    ret_upslope_24[:] = np.nan
                    ret_upslope_24[:] = ret_upslope[:]
                elif len(ret_upslope) != len(obs_diff[Date]):
                    ret_upslope_24 = np.empty(shape=(int(ret_upslope.shape[0]/60), ))
                    ret_upslope_24[:] = np.nan
                    for i in range(int(ret_upslope.shape[0]/60)):
                        ret_upslope_24[i] = np.sum(ret_upslope[i*60:(i+1)*60])
                del ret_upslope
                
                # assign precip values to easterly wind regime westerly wind regime will have nan
                ret_precip = np.empty(shape=ret_upslope_24.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_easterly[Date],] = ret_upslope_24[obs_easterly[Date],]

                ret_pulsed_24 = np.empty(shape=ret_upslope_24.shape)
                ret_pulsed_24[:] = np.nan
            if np.array(idx).shape[1] == 0:
                print([Date], 'no retried values')                    
                ret_upslope_24 = np.empty(shape=obs_diff[Date].shape)
                ret_upslope_24[:] = np.nan
                
                ret_pulsed_24 = np.empty(shape=obs_diff[Date].shape)
                ret_pulsed_24[:] = np.nan
                
                ret_precip = np.empty(shape=obs_diff[Date].shape)
                ret_precip[:] = np.nan
        
        elif len(obs_westerly[Date]) == 0 and len(obs_easterly[Date]) == 0:
            print([Date], 'no observations')
            idx = np.nan
            
            ret_upslope_24 = np.empty(shape=obs_diff[Date].shape)
            ret_upslope_24[:] = np.nan
            
            ret_pulsed_24 = np.empty(shape=obs_diff[Date].shape)
            ret_pulsed_24[:] = np.nan
            
            ret_precip = np.empty(shape=obs_diff[Date].shape)
            ret_precip[:] = np.nan
            
            ret_total_precip = np.nan
            ret_total_pulsed = np.nan
            ret_total_upslope = np.nan
        
        # assign sum values to days
        ret_total_precip = np.nansum(ret_precip)
        ret_total_pulsed = np.nansum(ret_pulsed_24[obs_westerly[Date], ])
        ret_total_upslope = np.nansum(ret_upslope_24[obs_easterly[Date], ])
        
        print([Date],'obs:', 'easterly:', np.round(obs_precip_east[Date], 2),
                      'westerly:', np.round(obs_precip_west[Date], 2),
                      'total:', np.round(obs_total_precip[Date], 2))
        print([Date],'ret:', 'easterly:', np.round(ret_total_upslope, 2),
                      'westerly:', np.round(ret_total_pulsed, 2),
                      'total:', np.round(ret_total_precip, 2))
        



    

        ### write netCDF file
        cF.createFolder(txt_dir+station+'/combined/')
        fw = netCDF4.Dataset(txt_dir+station+'/combined/precip_wind_temp_%s.nc' %(Date), 'w')
        ### create dimensions
        fw.createDimension('time', ret_precip.shape[0])
        fw.createDimension('24h_acc_sum', np.array([ret_total_precip]).shape[0])

        ret_upslope_w = get_netCDF_variable(fw, 'upslope_precip_amount', ret_upslope_24, ('time',))
        ret_pulsed_w  = get_netCDF_variable(fw, 'pulsed_precip_amount', ret_pulsed_24, ('time',))
        ret_precip_w  = get_netCDF_variable(fw, 'precip_amount', ret_precip, ('time',))
        ret_tot_up_w  = get_netCDF_variable(fw, '24h_upslope_sum', np.array([ret_total_upslope]), ('24h_acc_sum', ))
        ret_tot_pu_w  = get_netCDF_variable(fw, '24h_pulsed_sum',  np.array([ret_total_pulsed]), ('24h_acc_sum',))
        ret_tot_pp_w  = get_netCDF_variable(fw, '24h_precip_sum', np.array([ret_total_precip]), ('24h_acc_sum',))
        obs_ws_w      = get_netCDF_variable(fw, 'wind_speed_rolling_mean', obs_wind_speed_rolling[Date], ('time',) )
        obs_wd_w      = get_netCDF_variable(fw, 'wind_direction_rolling_mean', obs_wind_dir_rolling[Date], ('time',) )
        obs_at_w      = get_netCDF_variable(fw, 'air_temp_rolling_mean', obs_air_temp_rolling[Date], ('time',) )

        del ret_pulsed_24, ret_precip, ret_upslope_24, idx
        fw.close()

[20161108] no observations
[20161108] obs: easterly: nan westerly: nan total: nan
[20161108] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20161109] obs: easterly: 0.31 westerly: 0.0 total: 0.31
[20161109] ret: easterly: 1.08 westerly: 0.0 total: 1.08
[20161110] obs: easterly: 0.03 westerly: 0.09 total: 0.12
[20161110] ret: easterly: 0.15 westerly: 0.12 total: 0.27
[20161111] no observations
[20161111] obs: easterly: nan westerly: nan total: nan
[20161111] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20161112] obs: easterly: 3.45 westerly: 0.0 total: 3.45
[20161112] ret: easterly: 5.96 westerly: 0.0 total: 5.96
[20161113] obs: easterly: 1.11 westerly: 0.36 total: 1.47
[20161113] ret: easterly: 1.16 westerly: 0.09 total: 1.25
[20161114] obs: easterly: 2.22 westerly: 1.21 total: 3.43
[20161114] ret: easterly: 1.3 westerly: 0.28 total: 1.58
[20161115] obs: easterly: 2.38 westerly: 2.06 total: 4.44
[20161115] ret: easterly: 0.0 westerly: 0.34 total: 0.34
[20161116] obs: easterly: 5.91 weste

[20170116] obs: easterly: 0.0 westerly: 0.5 total: 0.5
[20170116] ret: easterly: 0.0 westerly: 0.27 total: 0.27
[20170117] obs: easterly: 0.0 westerly: 1.6 total: 1.6
[20170117] ret: easterly: 0.0 westerly: 0.05 total: 0.05
[20170118] obs: easterly: 0.0 westerly: 1.7 total: 1.7
[20170118] ret: easterly: 0.0 westerly: 0.04 total: 0.04
[20170119] obs: easterly: 0.0 westerly: 2.45 total: 2.45
[20170119] ret: easterly: 0.0 westerly: 0.01 total: 0.01
[20170120] obs: easterly: 0.0 westerly: 0.3 total: 0.3
[20170120] ret: easterly: 0.0 westerly: 0.1 total: 0.1
[20170121] obs: easterly: 0.23 westerly: 0.09 total: 0.32
[20170121] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170122] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170122] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170123] obs: easterly: 0.0 westerly: 1.1 total: 1.1
[20170123] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170124] obs: easterly: 0.0 westerly: 0.6 total: 0.6
[20170124] ret: easterly: 0.0 westerly: 0.01 total: 

[20170322] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170322] ret: easterly: 0.0 westerly: 0.02 total: 0.02
[20170323] obs: easterly: 0.0 westerly: 0.46 total: 0.46
[20170323] ret: easterly: 0.0 westerly: 0.34 total: 0.34
[20170324] obs: easterly: 0.0 westerly: 2.52 total: 2.52
[20170324] ret: easterly: 0.0 westerly: 1.02 total: 1.02
[20170325] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170325] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170326] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170326] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170327] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170327] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170328] obs: easterly: 0.0 westerly: 0.0 total: 0.0
[20170328] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170329] obs: easterly: 0.17 westerly: 0.01 total: 0.18
[20170329] ret: easterly: 0.0 westerly: 0.0 total: 0.0
[20170330] obs: easterly: 12.04 westerly: 4.46 total: 16.49
[20170330] ret: easterly: 11.78 westerly: 0.74 